# Converting PyTorch to TensorFlow Lite for xCORE Using ONNX

ONNX is an open format built to represent machine learning models. We can convert from PyTorch to ONNX, then from ONNX to TensorFlow, then from TensorFlow to TensorFlow Lite, and finally, run it through xformer to optimise it for xCORE.

Ensure that you have installed Python 3.8 and have the installed requirements.txt

In [1]:
import sys
import os

# allow importing helper functions from local module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Import PyTorch Model

For this example, we use YOLOv8.

In [ ]:
import torch
from ultralytics import YOLO

pytorch_yolo = YOLO("yolov8n")
pytorch_yolo.val()

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Ultralytics YOLOv8.0.146 🚀 Python-3.10.8 torch-1.12.0+cu102 CPU (Intel Core(TM) i5-1038NG7 2.00GHz)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients

Dataset 'coco.yaml' images not found ⚠️, missing path '/home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets/coco/val2017.txt'
Unzipping /home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets/coco2017labels-segments.zip to /home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets...
Skipping /home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets/coco2017labels-segments.zip unzip (already unzipped)
Unzipping /home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets/coco/images/val2017.zip to /home/jovyan/work/pytorch_to_tflite/YOLOv8/datasets/coco/images...
Skipping /home/jovyan/work/pytorch_to_tfli

## Prepare Datasets

YOLO is trained on the COCO dataset.

Once we have downloaded and extracted the dataset, we can create generators which returns a tuple of [path, original image, original shape, Tensor of type BCHW]

In [73]:
from PIL import Image
import numpy

def open_and_preprocess(path: str):
    size = pytorch_yolo.model.args["imgsz"]
    pil_img = Image.open("./coco/" + path.strip()).convert("RGB")
    resized_img = pil_img.resize((size, size))
    np_arr = numpy.array(resized_img).transpose([2, 0, 1])
    return (
        path,
        numpy.array(pil_img),
        numpy.array(pil_img).shape,
        torch.from_numpy(numpy.ascontiguousarray(numpy.expand_dims(np_arr, 0).astype(numpy.single) / 255))
    )

# generator that returns validation images as tuple [path, original image, original image shape, torch tensors of shape BCHW]
def validation_images():
    with open("./coco/val2017.txt") as fh:
        for path in fh.readlines():
            yield open_and_preprocess(path)
                
# generator that returns training images as tuple [path, original image, original image shape, torch tensors of shape BCHW]
def train_images():
    with open("./coco/train2017.txt") as fh:
        for path in fh.readlines():
            yield open_and_preprocess(path)


## Perform an infrence on the pytorch model
Perform inference on the model to see how it works.

In [ ]:
demo_image = next(validation_images())
# using yolo wrapper
results = pytorch_yolo(demo_image[3]))  # return a list of Results objects

In [ ]:
# using model directly
tensor = next(demo_image[3])
pytorch_results = pytorch_yolo.model(tensor)  # return a list of Results objects

In [26]:
# see what the type of the model is
type(pytorch_yolo.model)

ultralytics.nn.tasks.DetectionModel

In [13]:
"""
    https://github.com/ultralytics/ultralytics/blob/c3c27b019a9516a9b2c78c291b61ef7cf97ff7f3/ultralytics/engine/results.py#L66

    The class for holding results is Results, which takes instances of Boxes, Masks, Keypoints and Probs, which take tensors and process their values:
    
        boxes (torch.Tensor | numpy.ndarray): A tensor or numpy array containing the detection boxes,
        masks (torch.Tensor | np.ndarray): A tensor containing the detection masks, with shape (num_masks, height, width).
        keypoints (torch.Tensor | np.ndarray): A tensor containing the detection keypoints, with shape (num_dets, num_kpts, 2/3). 
        probs (torch.Tensor | np.ndarray): A tensor containing the detection keypoints, with shape (num_class, ).
"""
a, b = pytorch_results

In [24]:
# The first tensor has the same shape as output0 here: https://github.com/ultralytics/ultralytics/blob/c3c27b019a9516a9b2c78c291b61ef7cf97ff7f3/ultralytics/engine/exporter.py#L320
# but that should only be the case for DetectionModels, which should only have a single output. This model however has two outputs.
print(a.shape)

torch.Size([1, 84, 8400])
torch.Size([1, 144, 80, 80])
torch.Size([1, 144, 40, 40])
torch.Size([1, 144, 20, 20])


In [ ]:
from ultralytics.utils.ops import non_max_suppression
from ultralytics.engine.results import Results

preds = non_max_suppression(
    a,
    pytorch_yolo.predictor.args.conf,
    pytorch_yolo.predictor.args.iou,
    agnostic=pytorch_yolo.predictor.args.agnostic_nms,
    max_det=pytorch_yolo.predictor.args.max_det,
    classes=pytorch_yolo.predictor.args.classes
)
results = Results(
    orig_img = demo_image[1],
    path = demo_image[0],
    names = demo_image[2],
    boxes=preds
)


## Convert to ONNX


In [22]:
# This is only for shape info for tracing the model during conversion
sample_input = next(validation_images())

onnx_model_path = "yolov8_v2.onnx"

torch.onnx.export(
    pytorch_yolo.model,
    sample_input,
    onnx_model_path,
    input_names=['images'],
    output_names = ['output0']
)

/opt/conda/lib/python3.10/site-packages/ultralytics/nn/modules/head.py:50: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif self.dynamic or self.shape != shape:


### Check the exported model

In [23]:
import onnx
onnx_model = onnx.load(onnx_model_path)
onnx.checker.check_model(onnx_model)

### Check ONNX Output

### Convert ONNX to Keras
We do this using the `onnx2tf` package: https://github.com/PINTO0309/onnx2tf

### Check the conversion to keras

## Convert Keras to TFLite (int8)
We will still feed the data into the model in float32 format for convinence but the internals of the model will be int8. This will require representitive data but as we interface in float32 we can use the pytorch preprocessing. 

### Representative Dataset

To convert a model into to a TFLite flatbuffer, a representative dataset is required to help in quantisation. Refer to [Converting a keras model into an xcore optimised tflite model](https://colab.research.google.com/github/xmos/ai_tools/blob/develop/docs/notebooks/keras_to_xcore.ipynb) for more details on this.

# Analyse Models
Now the model is converted and we have confirmed that it works, let us take a look inside the converted models to see how good the conversion is.

## Check Operator Counts

Let us take a look at the operator counts inside the converted model. This uses a helper function defined in `../utils`, but this step is not necessary to convert the model.

## Compare Accuracy

Let's compare the accuracy of the converted model to the original PyTorch model.

To do this, we take a large sampel from imagenet_v2 and compare the classifications returned by the models.